In [9]:
from pymatgen.io.espresso.PWscf import PWxml, PWin
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.core import Spin
import xml.etree.ElementTree as ET
from monty.io import zopen
import xmltodict
import numpy as np

In [10]:
#filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_vcrelax.xml'
#filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_scf.xml'
filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_bands.xml'
#filename = 'pymatgen/io/espresso/tests/data/FeO_DFTU.xml'
#filename = 'pymatgen/io/espresso/tests/data/MoS2_vdw.xml'
#filename = 'pymatgen/io/espresso/tests/data/Ni_spin_pol.xml'
pwout = PWxml(filename)

In [11]:
vasprun = Vasprun('pymatgen/io/espresso/tests/data/Sr3PbO_vasprun.xml')

In [12]:
with open(filename, encoding="UTF-8") as xml:
    data = xmltodict.parse(xml.read())['qes:espresso']

In [13]:
data['output']['band_structure'].get('lowestUnoccupiedLevel', None)

'3.537375162118481e-1'

In [14]:
input_filename = 'pymatgen/io/espresso/tests/data/Ni_spin_pol.pwi'
pwin = PWin(input_filename)

In [15]:
pwin.atomic_species

In [16]:
pwin.electrons

OrderedDict([('diagonalization', 'david'),
             ('conv_thr', 1e-08),
             ('mixing_beta', 0.7)])

In [17]:
x = [None, None, {'hi': 2}]
if not all(x):
    print('Somrthing is wrong')

Somrthing is wrong


In [18]:
bool(x[2])

True

In [19]:
pwin.atomic_positions.items

[{'symbol': 'Ni', 'position': [0.0, 0.0, 0.0]}]

In [20]:
pwin.k_points.items

[{'k': [0.0, 0.0, 0.0], 'weight': 20, 'label': 'gamma'},
 {'k': [0.5, 0.5, 0.5], 'weight': 10, 'label': 'L'},
 {'k': [0.5, 0.25, 0.75], 'weight': 10, 'label': 'W'},
 {'k': [0.5, 0.0, 0.5], 'weight': 10, 'label': 'X'},
 {'k': [0.0, 0.0, 0.0], 'weight': 20, 'label': 'gamma'},
 {'k': [0.375, 0.375, 0.75], 'weight': 1, 'label': 'K'}]